In [1]:
import os
import pyowm
import pandas as pd
import numpy as np
import json
import requests
from pyowm.weatherapi25.parsers import forecastparser

from auth import(
    OWM_api_key)

In [2]:
weather_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
main_dir = os.path.abspath(os.path.join(weather_dir, os.pardir))

In [3]:
owm = pyowm.OWM(OWM_api_key)

In [4]:
park_loc = pd.DataFrame({
    'name': ['Prospect Park', 'Wooster Square'],
    'lat': [40.662551, 41.304745],
    'long': [-73.969256, -72.917757]})

In [5]:
park_name = 'Prospect Park'
park_lat = park_loc.loc[park_loc.name == park_name, 'lat'].iloc[0]
park_long = park_loc.loc[park_loc.name == park_name, 'long'].iloc[0]

park_fcstr = owm.three_hours_forecast_at_coords(
    float(park_lat),float(park_long))

In [6]:
OWM_forcast_cols = [
    'park_name', 'reception_time', 'reference_time', 'sunset_time',
    'sunrise_time', 'clouds', 'rain', 'snow', 'wind_speed',
    'wind_deg', 'humidity', 'press', 'press_sea', 'temp',
    'temp_feels', 'temp_max', 'temp_min', 'status', 'detailed_status']

In [7]:
park_forcast = park_fcstr.get_forecast()

In [8]:
def forcast_to_df(park_forcast, t_sunrise, t_sunset, tz_correct):
    ''' Takes a forcast object and sunrise/sunset time,
        Returns datafram of relevant info
    '''
    # Neet to adjust sunrise/set times, should only go
    # into columns of the same day (or just use last day in window?)
    
    forcast_df = pd.DataFrame(columns=OWM_forcast_cols)

    for x in park_forcast.get_weathers():
        if (t_sunrise == 0) or (t_sunset == 0):
            x_sunrise = np.nan
            x_sunset = np.nan
        else:
            x_sunrise = t_sunrise + tz_correct
            x_sunset = t_sunset + tz_correct

        x_rain = 0
        if x.get_rain():
            x_rain = x.get_rain()['3h']

        x_snow = 0
        if x.get_snow():
            x_snow = x.get_snow()['3h']

        to_add_dict = {
            'park_name': [park_name],
            'reception_time': [park_forcast.get_reception_time('unix') + tz_correct],
            'reference_time': [x.get_reference_time() + tz_correct],
            'sunrise_time': [x_sunrise],
            'sunset_time': [x_sunset],
            'clouds': [x.get_clouds()],
            'rain': [x_rain],
            'snow': [x_snow],
            'wind_speed': [x.get_wind(unit='meters_sec')['speed']],
            'wind_deg': [x.get_wind(unit='meters_sec')['deg']],
            'humidity': [x.get_humidity()],
            'press': [x.get_pressure()['press']],
            'press_sea': [x.get_pressure()['sea_level']],
            'temp': [x.get_temperature(unit='fahrenheit')['temp']],
            'temp_feels': [x.get_temperature(unit='fahrenheit')['feels_like']],
            'temp_max': [x.get_temperature(unit='fahrenheit')['temp_max']],
            'temp_min': [x.get_temperature(unit='fahrenheit')['temp_min']],
            'status': [x.get_status()],
            'detailed_status': [x.get_detailed_status()]}
        
        to_add = pd.DataFrame(to_add_dict)
        forcast_df = forcast_df.append(to_add, ignore_index=True)
    
    return forcast_df

In [9]:
forcast_df = forcast_to_df(park_forcast, 0, 0, 10)
forcast_df.head()

,park_name,reception_time,reference_time,sunset_time,sunrise_time,clouds,rain,snow,wind_speed,wind_deg,humidity,press,press_sea,temp,temp_feels,temp_max,temp_min,status,detailed_status
0,Prospect Park,1586899603,1586908810,NaN,NaN,92,0,0,4.96,214,55,1016,1016,55.27,46.69,55.27,51.44,Clouds,overcast clouds
1,Prospect Park,1586899603,1586919610,NaN,NaN,100,0.17,0,4.25,290,68,1017,1017,50.92,43.48,50.92,48.04,Rain,light rain
2,Prospect Park,1586899603,1586930410,NaN,NaN,100,0.5,0,4.51,283,66,1015,1015,48.24,39.85,48.24,46.33,Rain,light rain
3,Prospect Park,1586899603,1586941210,NaN,NaN,100,0.56,0,5.52,316,76,1016,1016,42.22,32.20,42.22,41.27,Rain,light rain
4,Prospect Park,1586899603,1586952010,NaN,NaN,100,0.32,0,6.84,346,63,1016,1016,42.03,29.61,42.03,42.03,Rain,light rain


Errors in pyowm means I can't get sunrise/sunset time directly, so moving to:

- Calling the api manually for the forcast JSON
- Extracting the sunrise/sunset time
- Passing the JSON back into the pyowm parser then use the work from above

In [10]:
api_url = 'http://api.openweathermap.org/data/2.5/forecast'
payload = {
    'lat': park_lat,
    'lon': park_long,
    'appid': OWM_api_key}
r = requests.get(api_url, params=payload)

In [11]:
d = json.loads(r.text)

In [12]:
sunrise_time = d['city']['sunrise']
sunset_time = d['city']['sunset']
tz_correct = d['city']['timezone']

In [13]:
weather_df = forcast_to_df(forecastparser.ForecastParser().parse_JSON(r.text), sunrise_time, sunset_time, tz_correct)

In [14]:
weather_df.head(5)

,park_name,reception_time,reference_time,sunset_time,sunrise_time,clouds,rain,snow,wind_speed,wind_deg,humidity,press,press_sea,temp,temp_feels,temp_max,temp_min,status,detailed_status
0,Prospect Park,1586885223,1586894400,1586892827,1586845059,92,0,0,4.96,214,55,1016,1016,55.29,46.71,55.29,51.44,Clouds,overcast clouds
1,Prospect Park,1586885223,1586905200,1586892827,1586845059,100,0.17,0,4.25,290,68,1017,1017,50.92,43.48,50.92,48.04,Rain,light rain
2,Prospect Park,1586885223,1586916000,1586892827,1586845059,100,0.5,0,4.51,283,66,1015,1015,48.25,39.87,48.25,46.33,Rain,light rain
3,Prospect Park,1586885223,1586926800,1586892827,1586845059,100,0.56,0,5.52,316,76,1016,1016,42.22,32.20,42.22,41.27,Rain,light rain
4,Prospect Park,1586885223,1586937600,1586892827,1586845059,100,0.32,0,6.84,346,63,1016,1016,42.03,29.61,42.03,42.03,Rain,light rain


The dataframe weather_df is what I'll be adding to the log of forcast data.

In [15]:
weather_df.to_csv('example_weather_df.csv')